# Applying Cosine Similarity Between TikToks and NYT Articles

In [1]:
import pandas as pd
import numpy as np
from numpy.linalg import norm
import string
import seaborn as sns 
import matplotlib.pyplot as plt
import os
import json
import datetime as dt
import time
import using_nyt_api as nyt

main_dir = os.path.dirname(os.getcwd())
analysis_dir = f'{main_dir}/analysis'
pyktok_dir = f'{main_dir}/pyktok-results'

## Cosine Similarity

In [2]:
def getVocabulary(textchunk):
    """Given some text, create the vocabulary of unique words."""
    textchunk = textchunk.lower()
    cleantext = "".join(char for char in textchunk if char not in string.punctuation)
    words = set(cleantext.split())
    voc = sorted(words)
    return voc

def text2vector(sentence, voc):
    """Given a sentence and the vocabulary for the problem,
    turn every sentence into a vector.
    """
    cleantext = "".join(char for char in sentence if char not in string.punctuation)
    words = cleantext.lower().split()
    vector = [words.count(w) for w in voc]
    return vector

def cosineSimilarity(vec1,vec2):
    """Calculate the cosine similarity between two vectors."""
    V1 = np.array(vec1)
    V2 = np.array(vec2)
    cosine = np.dot(V1,V2)/(norm(V1)*norm(V2))
    return cosine

def rankDocuments(query,sentences):
    """Given a query and some sentences, rank the sentences for
    which are the most similar to the query"""
    # Step 1: create vocabulary
    voc = getVocabulary(" ".join(sentences))

    # Step 2: generate vector for query
    queryVec = text2vector(query,voc)

    # Step 3: generate vector for sentences and calculate cosine similarity at once
    similarities = []
    for sent in sentences:
        sentVec = text2vector(sent,voc)
        sim = cosineSimilarity(queryVec,sentVec)
        similarities.append((round(sim,4),sent)) # keep track of sentences

    similarities.sort(reverse=True)
    return similarities # a tuple of (cosine similarity, sentence)

## NYTimes API

In [3]:
def check_file_exists(filepath,filename):
    if filename in os.listdir(filepath):
        return True
    return False

def get_nyt_data(date,key,save_csv):
    '''Takes a date, NYT API key, and boolean for whether to save data as csv. 
    Returns a df containing the abstract, lead_paragraph, pub_date, 
    document_type, section_name, type_of_material, headline, and keywords 
    of articles for that date and saves df to {date}-articles.csv.'''
    # check if articles for given date exist
    filepath = f'{analysis_dir}/nyt_data/' # save files to analysis/nyt_data folder
    filename = f'{date}-articles.csv'
    file_exists = check_file_exists(filepath,filename)
    
    if file_exists: # reads df from existing file if file exists
        nyt_data = pd.read_csv(f'{filepath}/{filename}')
    else: # gets articles by calling NYT API and creates df
        articles = nyt.get_articles_by_date(date,key)
        nyt_data = nyt.create_df(articles,date,filepath,write_csv=save_csv)
    return nyt_data

In [105]:
# Testing on one date
date = '2024-03-12'
key = '1DFmIMxxqdYl8wJBPqAFxtHkimk86Qtn'
save_csv = True
nyt_data = get_nyt_data(date,key,save_csv)
nyt_data.head(3)

,Unnamed: 0,abstract,lead_paragraph,pub_date,document_type,section_name,type_of_material,headline,keywords
0,0,The Republican Party apparatus is undergoing a...,Days after allies took over the Republican Nat...,2024-03-12T00:03:05+0000,article,U.S.,News,"Trump Aides, Taking Over R.N.C., Order Mass La...",Republican National Committee;Republican Party...
1,1,ABC’s telecast of the 96th Academy Awards on S...,The comeback of live event TV continues.,2024-03-12T00:09:17+0000,article,Business Day,News,"‘Barbenheimer,’ and an Early Start, Boost Osca...",Academy Awards (Oscars);Ratings (Audience Meas...
2,2,People spit out the word “Zionist” when they r...,"Every time I write, as I did last week, that I...",2024-03-12T00:15:17+0000,article,Opinion,Op-Ed,Where Antisemitism and Anti-Zionism Collide,Israel;Gaza Strip;Zionism;Palestinians;Jews an...


## Tiktoks with News Hashtags

In [107]:
def get_pyktok_metadata_by_code(code): # NOT USING
    '''Returns a df of metadata collected from pyktok for the given code'''
    filename = f'{pyktok_dir}/results_{code}.csv'
    print(f'Getting pyktok data from file {filename}')
    metadata = pd.read_csv(filename)
    return metadata

def get_news_vids_by_code(code): # NOT USING
    '''Get the videos with news hashtags based on code'''
    filename = f'{analysis_dir}/videos-newsHashtags-{code}.json'
    print(f'Getting news-related videos from file {filename}')
    with open(filename, 'r') as inFile:
        data = json.load(inFile)
    news_vids = pd.read_json(data, orient="split")
    return news_vids

def add_date_to_news_vids(metadata,news_vids):
    updated_df = news_vids.merge(metadata[['video_id','video_timestamp']], on='video_id', how='left') # keeps all ids in news_vids
    updated_df['date'] = updated_df['video_timestamp'].astype(str).apply(lambda timestamp: timestamp[:10])
    return updated_df


def get_pyktok_metadata_by_filename(filename):
    '''Returns a df of metadata collected from pyktok for the given file'''
    file = f'{pyktok_dir}/{filename}'
    print(f'Getting pyktok data from file {file}')
    metadata = pd.read_csv(file)
    return metadata

def get_news_vids_by_filename(filename):
    '''Get the videos with news hashtags based on file'''
    file = f'{analysis_dir}/{filename}'
    print(f'Getting news-related videos from file {file}')
    with open(file, 'r') as inFile:
        data = json.load(inFile)
    news_vids = pd.read_json(data, orient="split")
    return news_vids

In [109]:
# # Test for code 12345
# code = 12345

# # Pyktok Metadata is located within pyktok-results folder
# metadata = get_pyktok_metadata_by_code(code)
# print(f'Metadata DF Shape: {metadata.shape}')
# metadata.head(3)

# # JSON files containing news-related videos located in analysis folder
# news_vids = get_news_vids_by_code(code)
# print(f'Shape of News-related Videos DF: {news_vids.shape}')
# news_vids.head(3)

# # Add video_timestamp from metadata df to news_vids df
# updated_news_vids = add_date_to_news_vids(metadata,news_vids)
# updated_news_vids.head(3)

## Cosine Similarity between some TikToks and some NYT articles

Each of your transcribed videos should have both the date and the various text features (transcript, description, hashtags, or suggested words). Ideally, we will create embeddings of each of them separately and even an embedding of transcript + description + suggested words (hashtags are already in the description) and then compare each of them against the respective NYT headlines of the day. 

However, it is okay if you pick only one category to get the embedding and calculate the cosine similarity. 
For each post, save the most similar NYT article and the cosine score. Put everything in a dataframe so that you can sort to see the most similar pairs and the least similar ones, and also use describe() to see the range of cosine similarity scores. 

For each post, save the most similar NYT article and the cosine score. Put everything in a dataframe so that you can sort to see the most similar pairs and the least similar ones, and also use describe() to see the range of cosine similarity scores.

In [86]:
def get_all_nyt_data(dates):
    '''Saves and returns cleaned dfs for each date'''
    nyt_dfs = []
    for date in dates:
        df = get_nyt_data(date,key,True)
        print(f'\nShape of df for {date}: {df.shape}')
        nyt_dfs.append(df)
    return nyt_dfs

In [78]:
def apply_cosine_similarity_by_feature(news_vids,video_id,key,feature):
    '''Takes df of videos, video id (as int64), NYT API key, and an feature
    of the news_vids df (suggested_words, video_description, etc), and finds the 
    cosine similarity between that attribute and all the NYTimes headlines 
    from the date the video was watched.'''
    # Get the relevant tiktok video and its video description
    video = news_vids[news_vids['video_id']==video_id].iloc[0] # get 1 video
    feature_text = video[feature] # gets feature, such as description, suggested words, etc
    date = video['date']

    # Load the relevant nytdata for given date
    nyt_filepath = f'{analysis_dir}/nyt_data'
    nyt_filename = f'{date}-articles.csv'
    print(f'Looking for {nyt_filepath}/{nyt_filename}')
    
    try:
        nyt_df = pd.read_csv(f'{nyt_filepath}/{nyt_filename}')
        print(f'Loaded data from {nyt_filename}')
    except FileNotFoundError as fnfe:
        print(fnfe)
        print(f'File {nyt_filename} not found. Creating it instead.')
        nyt_df = get_nyt_data(date,key,True)

    # Get all the headlines for the date
    try:
        headlines = nyt_df['headline'].values
    except KeyError:
        print("\'headline\' not found in nyt_df.")
        headlines = []

    # Apply cosine similarity
    similarities = rankDocuments(feature_text,headlines) # a tuple of (cosine similarity, sentence)
    return similarities, feature_text, date

In [83]:
%%time
# EXAMPLE FOR ONE VIDEO ID
code = 12345
metadata_file = 'results_12345(19019 rows).csv'
news_vids_file = 'videos-newsHashtags-12345.json'
# metadata = get_pyktok_metadata_by_code(code)
# news_vids = get_news_vids_by_code(code)
metadata = get_pyktok_metadata_by_filename(metadata_file)
news_vids = get_news_vids_by_filename(news_vids_file)
updated_news_vids = add_date_to_news_vids(metadata,news_vids)
video_id = updated_news_vids['video_id'].values[0] # get a particular video id
feature = 'video_description' #'suggested_words' 

similarities,feature_text,date = apply_cosine_similarity_by_feature(updated_news_vids,video_id,key,feature)

print(f'\nTikTok User Code: {code}')
print(f'\nVideo ID: {video_id}')
print(f'\nVideo\'s {feature}: {feature_text}')
print(f'\nDate watched: {date}')
print(f'\nTop 5 similarities:')
for sim in similarities[:5]:
    print(sim)

Getting pyktok data from file /Users/edithpo/Downloads/CS315_Project2_Group1/pyktok-results/results_12345(19019 rows).csv
Getting news-related videos from file /Users/edithpo/Downloads/CS315_Project2_Group1/analysis/videos-newsHashtags-12345.json
Looking for /Users/edithpo/Downloads/CS315_Project2_Group1/analysis/nyt_data/2024-03-05-articles.csv
Loaded data from 2024-03-05-articles.csv

TikTok User Code: 12345

Video ID: 7343058460025703722

Video's video_description: NBC News and MSNBC Correspondent Shaquille Brewster speaks to a North Carolina Trump voter who would not consider voting for Nikki Haley because she's a woman. "A woman’s not gonna be a good president," he tells Brewster, "All a woman's good for in my book is having babies and taking care of the house." #fyp #foryou #politicaltiktok #politics #2024president #election2024 #trump #haley #vote 

Date watched: 2024-03-05

Top 5 similarities:
(0.4616, 'The Lindbergh Baby Kidnapping: A Grisly Theory and a Renewed Debate')
(0.39

/var/folders/yl/zl57ck115t9_8v83kxw1gtc40000gn/T/ipykernel_50696/1360562106.py:36: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  news_vids = pd.read_json(data, orient="split")


In [84]:
def get_all_cosine_similarities_by_feature(updated_news_vids,video_ids,key,feature):
    results = []

    for video_id in video_ids:
        print(f'Video ID: {video_id}')
        try:
            result_dict = {}
            similarities,feature_text,date = apply_cosine_similarity_by_feature(updated_news_vids,video_id,key,feature)
            result_dict['video_id'] = video_id
            result_dict['date'] = date
            result_dict[f'{feature}'] = feature_text
            result_dict['similarities'] = similarities
            cos_sim_scores = [sim[0] for sim in similarities]
            nyt_headlines = [sim[1] for sim in similarities]
            result_dict['cos_sim_scores'] = cos_sim_scores
            result_dict['nyt_headlines'] = nyt_headlines
            results.append(result_dict)
        except TypeError as te:
            print(f'Problem with getting the cosine similarities for video {video_id}')
            print(te)

    return results

## Putting Everything Together

In [85]:
key = '1DFmIMxxqdYl8wJBPqAFxtHkimk86Qtn'

In [110]:
def get_results_by_key(key,code,feature):
    # Get data for given code
    metadata = get_pyktok_metadata_by_code(code)
    news_vids = get_news_vids_by_code(code)
    updated_news_vids = add_date_to_news_vids(metadata,news_vids)
    video_ids = updated_news_vids['video_id'].values 

    results = get_all_cosine_similarities_by_feature(updated_news_vids,video_ids,key,feature)
    df = pd.DataFrame(results)
    
    # Save results
    filename = f'cosine_sim_{feature}_{code}.csv'
    df.to_csv(filename)
    
    return df

def get_results_by_filenames(metadata_file,news_vid_file,feature):
    # Get data for given files
    metadata = get_pyktok_metadata_by_filename(metadata_file)
    news_vids = get_news_vids_by_filename(news_vid_file)
    updated_news_vids = add_date_to_news_vids(metadata,news_vids)
    video_ids = updated_news_vids['video_id'].values 

    results = get_all_cosine_similarities_by_feature(updated_news_vids,video_ids,key,feature)
    df = pd.DataFrame(results)
    
    # Save results
    filename = f'cosine_sim_{feature}_{code}.csv'
    df.to_csv(filename)
    
    return df

# print(f'\nTikTok User Code: {code}')
# print(f'\nVideo ID: {video_id}')
# print(f'\nVideo\'s {feature}: {feature_text}')
# print(f'\nDate watched: {date}')
# print(f'\nTop 5 similarities:')
# for sim in similarities[:5]:
#     print(sim)

### Account 12345

In [112]:
# code = 12345
# feature = "suggested_words"#"video_description" 
# df = get_results_by_key(key,code,feature)

# Using functions to retrieve data by filename instead of code
metadata_file = 'results_12345(19019 rows).csv'
news_vid_file = 'videos-newsHashtags-12345.json'
feature = 'suggested_words'#'video_description'
df = get_results_by_filenames(metadata_file,news_vid_file,feature)
df

Getting pyktok data from file /Users/edithpo/Downloads/CS315_Project2_Group1/pyktok-results/results_12345(19019 rows).csv
Getting news-related videos from file /Users/edithpo/Downloads/CS315_Project2_Group1/analysis/videos-newsHashtags-12345.json
Video ID: 7343058460025703722
Looking for /Users/edithpo/Downloads/CS315_Project2_Group1/analysis/nyt_data/2024-03-05-articles.csv
Loaded data from 2024-03-05-articles.csv


/var/folders/yl/zl57ck115t9_8v83kxw1gtc40000gn/T/ipykernel_50696/4242591404.py:36: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  news_vids = pd.read_json(data, orient="split")
/var/folders/yl/zl57ck115t9_8v83kxw1gtc40000gn/T/ipykernel_50696/1891048485.py:22: RuntimeWarning: invalid value encountered in scalar divide
  cosine = np.dot(V1,V2)/(norm(V1)*norm(V2))


Video ID: 7340809195807132971
Looking for /Users/edithpo/Downloads/CS315_Project2_Group1/analysis/nyt_data/2024-02-28-articles.csv
Loaded data from 2024-02-28-articles.csv
Video ID: 7341877699629108523
Looking for /Users/edithpo/Downloads/CS315_Project2_Group1/analysis/nyt_data/2024-03-02-articles.csv
Loaded data from 2024-03-02-articles.csv
Video ID: 7341485301010877728
Looking for /Users/edithpo/Downloads/CS315_Project2_Group1/analysis/nyt_data/2024-03-01-articles.csv
Loaded data from 2024-03-01-articles.csv
Video ID: 7341694662866046254
Looking for /Users/edithpo/Downloads/CS315_Project2_Group1/analysis/nyt_data/2024-03-02-articles.csv
Loaded data from 2024-03-02-articles.csv
Video ID: 7331462452195970337
Looking for /Users/edithpo/Downloads/CS315_Project2_Group1/analysis/nyt_data/2024-02-03-articles.csv
Loaded data from 2024-02-03-articles.csv
Problem with getting the cosine similarities for video 7331462452195970337
'NoneType' object is not iterable
Video ID: 7340363649212550405
L

Video ID: 7337716616479788331
Looking for /Users/edithpo/Downloads/CS315_Project2_Group1/analysis/nyt_data/2024-02-20-articles.csv
Loaded data from 2024-02-20-articles.csv
Video ID: 7337716616479788331
Looking for /Users/edithpo/Downloads/CS315_Project2_Group1/analysis/nyt_data/2024-02-20-articles.csv
Loaded data from 2024-02-20-articles.csv
Video ID: 7338131827300846890
Looking for /Users/edithpo/Downloads/CS315_Project2_Group1/analysis/nyt_data/2024-02-21-articles.csv
Loaded data from 2024-02-21-articles.csv
Video ID: 7338131827300846890
Looking for /Users/edithpo/Downloads/CS315_Project2_Group1/analysis/nyt_data/2024-02-21-articles.csv
Loaded data from 2024-02-21-articles.csv
Video ID: 7338586099721309471
Looking for /Users/edithpo/Downloads/CS315_Project2_Group1/analysis/nyt_data/2024-02-22-articles.csv
Loaded data from 2024-02-22-articles.csv
Video ID: 7338586099721309471
Looking for /Users/edithpo/Downloads/CS315_Project2_Group1/analysis/nyt_data/2024-02-22-articles.csv
Loaded da

Loaded data from 2024-02-07-articles.csv
Video ID: 7332797813983677717
Looking for /Users/edithpo/Downloads/CS315_Project2_Group1/analysis/nyt_data/2024-02-07-articles.csv
Loaded data from 2024-02-07-articles.csv
Video ID: 7332174452706397486
Looking for /Users/edithpo/Downloads/CS315_Project2_Group1/analysis/nyt_data/2024-02-05-articles.csv
Loaded data from 2024-02-05-articles.csv
Video ID: 7331850550503329066
Looking for /Users/edithpo/Downloads/CS315_Project2_Group1/analysis/nyt_data/2024-02-04-articles.csv
Loaded data from 2024-02-04-articles.csv
Video ID: 7328492050683333921
Looking for /Users/edithpo/Downloads/CS315_Project2_Group1/analysis/nyt_data/2024-01-26-articles.csv
Loaded data from 2024-01-26-articles.csv
Video ID: 7330744742084431147
Looking for /Users/edithpo/Downloads/CS315_Project2_Group1/analysis/nyt_data/2024-02-01-articles.csv
Loaded data from 2024-02-01-articles.csv
Video ID: 7331832637562047787
Looking for /Users/edithpo/Downloads/CS315_Project2_Group1/analysis/n

...

/var/folders/yl/zl57ck115t9_8v83kxw1gtc40000gn/T/ipykernel_50696/1891048485.py:22: RuntimeWarning: invalid value encountered in scalar divide
  cosine = np.dot(V1,V2)/(norm(V1)*norm(V2))


Loaded data from 2023-09-15-articles.csv
Problem with getting the cosine similarities for video 7279046355992071457
'NoneType' object is not iterable
Video ID: 7279704894506945798
Looking for /Users/edithpo/Downloads/CS315_Project2_Group1/analysis/nyt_data/2023-09-17-articles.csv
Loaded data from 2023-09-17-articles.csv
Problem with getting the cosine similarities for video 7279704894506945798
'NoneType' object is not iterable
Video ID: 7280016453565156650
Looking for /Users/edithpo/Downloads/CS315_Project2_Group1/analysis/nyt_data/2023-09-18-articles.csv
Loaded data from 2023-09-18-articles.csv
Problem with getting the cosine similarities for video 7280016453565156650
'NoneType' object is not iterable
Video ID: 7279568128894340395
Looking for /Users/edithpo/Downloads/CS315_Project2_Group1/analysis/nyt_data/2023-09-16-articles.csv
Loaded data from 2023-09-16-articles.csv
Video ID: 7280455645134966048
Looking for /Users/edithpo/Downloads/CS315_Project2_Group1/analysis/nyt_data/2023-09-1

Video ID: 7270139607281388833
Looking for /Users/edithpo/Downloads/CS315_Project2_Group1/analysis/nyt_data/2023-08-22-articles.csv
Loaded data from 2023-08-22-articles.csv
Problem with getting the cosine similarities for video 7270139607281388833
'NoneType' object is not iterable
Video ID: 7262387122319953195
Looking for /Users/edithpo/Downloads/CS315_Project2_Group1/analysis/nyt_data/2023-08-01-articles.csv
Loaded data from 2023-08-01-articles.csv
Problem with getting the cosine similarities for video 7262387122319953195
'NoneType' object is not iterable
Video ID: 7273441792849218848
Looking for /Users/edithpo/Downloads/CS315_Project2_Group1/analysis/nyt_data/2023-08-31-articles.csv
Loaded data from 2023-08-31-articles.csv
Problem with getting the cosine similarities for video 7273441792849218848
'NoneType' object is not iterable
Video ID: 7277231973263789355
Looking for /Users/edithpo/Downloads/CS315_Project2_Group1/analysis/nyt_data/2023-09-10-articles.csv
Loaded data from 2023-09-1

/var/folders/yl/zl57ck115t9_8v83kxw1gtc40000gn/T/ipykernel_50696/1891048485.py:22: RuntimeWarning: invalid value encountered in scalar divide
  cosine = np.dot(V1,V2)/(norm(V1)*norm(V2))


,video_id,date,suggested_words,similarities,cos_sim_scores,nyt_headlines
0,7343058460025703722,2024-03-05,"haley, nbc, nicki halley","[(nan, Trump Is Running on Dystopian Fantasies...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[Trump Is Running on Dystopian Fantasies, Micr..."
1,7340809195807132971,2024-02-28,"Rebecca Ferguson, hugh jackman, rebecca fergus...","[(0.2236, E-Bike Battery Caused Fire That Kill...","[0.2236, 0.2041, 0.1826, 0.0, 0.0, 0.0, 0.0, 0...",[E-Bike Battery Caused Fire That Killed Young ...
2,7341877699629108523,2024-03-02,"rhode island, wyoming state, 50 state song, ar...","[(0.4243, Scenes From the Smokehouse Creek Fir...","[0.4243, 0.3536, 0.3207, 0.291, 0.2828, 0.2774...",[Scenes From the Smokehouse Creek Fire in Texa...
3,7341485301010877728,2024-03-01,"Wendy Williams, denial is a river in egypt, we...","[(0.3105, If You Had $1 Billion to Give Away, ...","[0.3105, 0.27, 0.252, 0.2224, 0.2224, 0.1782, ...","[If You Had $1 Billion to Give Away, What Char..."
4,7341694662866046254,2024-03-02,"royal family news, kate middleton brother, lad...","[(0.3015, News Leaders Around the World Pledge...","[0.3015, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",[News Leaders Around the World Pledge Support ...
...,...,...,...,...,...,...
235,7278032406232223019,2023-09-12,"the fault in our stars, The Fault In Our Stars...","[(0.2914, Football Season), (0.2899, N.F.L. Re...","[0.2914, 0.2899, 0.2485, 0.2203, 0.2203, 0.194...","[Football Season, N.F.L. Reporter Files Racial..."
236,7277248227693333802,2023-09-10,"margaritaville, trump ai singing, margaritavil...","[(0.2585, Covid Hero or ‘Lockdown Ron’? DeSant...","[0.2585, 0.1819, 0.1715, 0.0, 0.0, 0.0, 0.0, 0...",[Covid Hero or ‘Lockdown Ron’? DeSantis and Tr...
237,7277312227831549230,2023-09-10,"college, College Student, College Life, Time-L...","[(0.3706, Russia’s ‘Merchant of Death’ Is Look...","[0.3706, 0.239, 0.2224, 0.1945, 0.1429, 0.138,...",[Russia’s ‘Merchant of Death’ Is Looking to Fo...
238,7274906751294524704,2023-09-04,"history, History Tiktok, jewish, Middle Easter...","[(nan, Lea Michele Ends ‘Dream’ Run in ‘Funny ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",[Lea Michele Ends ‘Dream’ Run in ‘Funny Girl’ ...


### Account 10824

In [ ]:
# Use get results by filename

### Account 50405

In [ ]:
# Use get results by filename